In [1]:
import pandas as pd

data = pd.read_csv('twitter_data_2020_12_20_2021_01_20.csv', parse_dates=['created_at'])
data = data.sort_values(by='created_at')
data.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,6.486600e+04,6.486600e+04,64866.0,6.486600e+04,64866.000000,64866.000000,64866.000000,64866.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.346076e+18,1.345947e+18,-500.0,8.438913e+17,1.159899,2.142787,4.471356,0.441017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.821845e+15,6.173234e+15,0.0,5.447926e+17,21.636357,64.337961,45.147545,0.496513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.340522e+18,3.326980e+17,-500.0,3.450300e+04,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.343731e+18,1.343688e+18,-500.0,3.183457e+09,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.346770e+18,1.346763e+18,-500.0,1.102851e+18,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.347417e+18,1.347374e+18,-500.0,1.290123e+18,0.000000,0.000000,2.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.352269e+18,1.352269e+18,-500.0,1.352154e+18,1513.000000,5552.000000,3242.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
null_features_counts = list(filter(lambda x: next(iter(x.values())), 
                        map(lambda f: {f: len(data.loc[data[f].isnull()])}, data.columns)))
print(str(null_features_counts))

[{'place': 64865}, {'quote_url': 58551}, {'thumbnail': 36259}, {'near': 64866}, {'geo': 64866}, {'source': 64866}, {'user_rt_id': 64866}, {'user_rt': 64866}, {'retweet_id': 64866}, {'retweet_date': 64866}, {'translate': 64866}, {'trans_src': 64866}, {'trans_dest': 64866}]


In [3]:
tweets = data.pivot_table(index=['tweet'], aggfunc ='size').to_frame()
tweets_dups = tweets.loc[tweets[0] > 1]
print('Unique tweets: ' + str(len(tweets.index)))
print('tweets duplicated: ' + str(len(tweets_dups.index)))
print('tweets duplicate rate: ' + str(len(tweets_dups.index) / len(tweets.index)))

Unique tweets: 48956
tweets duplicated: 10183
tweets duplicate rate: 0.2080031048288259


In [8]:
no_retweets = data.loc[data['retweets_count'] == 0]
duplicate_tweets = (no_retweets.groupby(['username', 'tweet']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')
   .drop_duplicates(subset='tweet'))

duplicate_tweets = duplicate_tweets.loc[duplicate_tweets['count'] > 1]
banned_usernames = duplicate_tweets['username']

clean_data = data[~data['username'].isin(banned_usernames)]

no_retweets = clean_data.loc[clean_data['retweets_count'] == 0]
duplicate_tweets = (no_retweets.groupby(['username', 'tweet']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')
   .drop_duplicates(subset='tweet'))

duplicate_tweets = duplicate_tweets.loc[duplicate_tweets['count'] > 1]
banned_usernames = duplicate_tweets['username']

clean_data = clean_data[~clean_data['username'].isin(banned_usernames)]

print(clean_data.loc[clean_data['retweets_count'] > 0].describe())

# clean_data.to_csv('twitter_data_clean2.csv')


                        id      conversation_id          created_at  \
12176  1340558190652506112  1340558190652506112 2020-12-20 02:22:33   
24208  1340558190652506112  1340558190652506112 2020-12-20 02:22:33   
64835  1340558190652506112  1340558190652506112 2020-12-20 02:22:33   
12131  1340590202562080768  1340542774454587392 2020-12-20 04:29:45   
64790  1340590202562080768  1340542774454587392 2020-12-20 04:29:45   

             date      time  timezone              user_id     username  \
12176  2020-12-20  02:22:33      -500             15061990     tjeezyyy   
24208  2020-12-20  02:22:33      -500             15061990     tjeezyyy   
64835  2020-12-20  02:22:33      -500             15061990     tjeezyyy   
12131  2020-12-20  04:29:45      -500  1277617350745796610  bittradebtc   
64790  2020-12-20  04:29:45      -500  1277617350745796610  bittradebtc   

                                           name place  ... geo source  \
12176  jeezy, a love story 🤰 tokennotneeded.com  

In [138]:
tweets = clean_data.pivot_table(index=['tweet'], aggfunc ='size').to_frame()
tweets_dups = tweets.loc[tweets[0] > 1]
print('Unique tweets: ' + str(len(tweets.index)))
print('tweets duplicated: ' + str(len(tweets_dups.index)))
print('tweets duplicate rate: ' + str(len(tweets_dups.index) / len(tweets.index)))

Unique tweets: 25924
tweets duplicated: 558
tweets duplicate rate: 0.021524456102453324


In [139]:
clean_data = clean_data.drop_duplicates(subset='tweet', keep="last")
clean_data.to_csv('twitter_data_clean3.csv')

In [188]:
clean_data2 = pd.read_csv('twitter_data_clean2.csv', parse_dates=['created_at'])
clean_data2 = clean_data2.sort_values(by='created_at')
clean_data2.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,12176,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,24208,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,64835,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,12131,1340590202562080768,1340542774454587392,2020-12-20 04:29:45,2020-12-20,04:29:45,-500,1277617350745796610,bittradebtc,Bittrade,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,64790,1340590202562080768,1340542774454587392,2020-12-20 04:29:45,2020-12-20,04:29:45,-500,1277617350745796610,bittradebtc,Bittrade,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [189]:
clean_data1 = pd.read_csv('twitter_data_clean.csv', parse_dates=['created_at'])
clean_data1 = clean_data2.sort_values(by='created_at')
clean_data1.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,12176,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,24208,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,64835,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,12131,1340590202562080768,1340542774454587392,2020-12-20 04:29:45,2020-12-20,04:29:45,-500,1277617350745796610,bittradebtc,Bittrade,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,64790,1340590202562080768,1340542774454587392,2020-12-20 04:29:45,2020-12-20,04:29:45,-500,1277617350745796610,bittradebtc,Bittrade,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [190]:
clean_data3 = pd.read_csv('twitter_data_clean3.csv', parse_dates=['created_at'])
clean_data3 = clean_data3.sort_values(by='created_at')
clean_data3.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,64835,1340558190652506112,1340558190652506112,2020-12-20 02:22:33,2020-12-20,02:22:33,-500,15061990,tjeezyyy,"jeezy, a love story 🤰 tokennotneeded.com",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,24163,1340590202562080768,1340542774454587392,2020-12-20 04:29:45,2020-12-20,04:29:45,-500,1277617350745796610,bittradebtc,Bittrade,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,24035,1340616069153771521,1340616069153771521,2020-12-20 06:12:32,2020-12-20,06:12:32,-500,1624918207,deribotinfo,DeriBot.info,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,12100,1340617072448069632,1340617072448069632,2020-12-20 06:16:32,2020-12-20,06:16:32,-500,1624918207,deribotinfo,DeriBot.info,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,64728,1340637885326741504,1340637885326741504,2020-12-20 07:39:14,2020-12-20,07:39:14,-500,2231159425,thepixiepost,Jenn Nieto 🐝I ♥️ CTP & HIVE🐝,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
